In [1]:
#### 패키지 임포트
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import os
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
pd.set_option('display.max_columns', None)

In [2]:
#### 본문to형태소(morphs) 파일
df = pd.read_csv("../dataset/2022/본문_형태소분석/나무문화_본문_2월_형태소분석.txt", encoding='UTF-8')
df['morphs']=df['morphs'].apply(lambda x : x.strip("'][").split("', '"))
print(df.shape, df.columns)
docs = df['morphs'].to_list()
print(len(docs))

(700406, 4) Index(['split_str', 'org_idx', 'morphs', 'pos_tag'], dtype='object')
700406


In [3]:
df

,split_str,org_idx,morphs,pos_tag
0,집 앞에 바로 도서관이 있었는데 이사 오고 1년이 다 돼서야 알게 된 거 있지 구석...,0,"[집, 앞, 에, 바로, 도서관, 이, 있었는데, 이사, 오고, 1년, 이, 다, ...","[('집', 'Noun'), ('앞', 'Noun'), ('에', 'Josa'), ..."
1,우웅 기준 오빠 목성의 위성 에우로파에는 관심없지만 이런 사랑이라면 훌훌 떠나도 좋...,0,"[우웅, 기준, 오빠, 목성, 의, 위성, 에우로파, 에는, 관심, 없지만, 이런,...","[('우웅', 'Exclamation'), ('기준', 'Noun'), ('오빠',..."
2,우리반 임시 시간표인데 월요일 1교시 문학 임시라니까 언제까지 이러는 건지는 모르겠...,0,"[우리, 반, 임시, 시간표, 인데, 월요일, 1, 교시, 문학, 임시, 라니까, ...","[('우리', 'Noun'), ('반', 'Noun'), ('임시', 'Noun')..."
3,약간 학교를 가고 싶기도 안 가고 싶기도 함,0,"[약간, 학교, 를, 가고, 싶기도, 안, 가고, 싶기도, 함]","[('약간', 'Noun'), ('학교', 'Noun'), ('를', 'Josa')..."
4,물론 내 의사와는 상관없겠지만 껄껄 내 사랑 초코나무숲이 판매 종료 한다는 소식에 ...,0,"[물론, 내, 의사, 와는, 상관없겠지만, 껄껄, 내, 사랑, 초코, 나무숲, 이,...","[('물론', 'Adverb'), ('내', 'Noun'), ('의사', 'Noun..."
...,...,...,...,...
700401,설날 작사 작곡 윤극영 노래 가사 쓰기 재생 12 좋아요 3 00 00 01 07 ...,13352,"[설날, 작사, 작곡, 윤극영, 노래, 가사, 쓰기, 재생, 12, 좋아요, 3, ...","[('설날', 'Noun'), ('작사', 'Noun'), ('작곡', 'Noun'..."
700402,아동문학가이자 동요작곡가 그리고 아동문화운동가셨던 윤극영님은 푸른 하늘 은하수 하얀...,13352,"[아동문학, 가, 이자, 동요, 작곡가, 그리고, 아동, 문화, 운동, 가셨던, 윤...","[('아동문학', 'Noun'), ('가', 'Josa'), ('이자', 'Noun..."
700403,일제강점기 중 우리나라 가사를 붙인 노래들을 작곡하여 어린이들에게 민족에 대한 자긍...,13352,"[일제강점기, 중, 우리나라, 가사, 를, 붙인, 노래, 들, 을, 작곡, 하여, ...","[('일제강점기', 'Noun'), ('중', 'Noun'), ('우리나라', 'N..."
700404,새해 복 많이 받으세요,13352,"[새해, 복, 많이, 받으세요]","[('새해', 'Noun'), ('복', 'Noun'), ('많이', 'Adverb..."


In [4]:
#### 각각의 본문이 str타입('나는 어제 밥을 먹었다')일 경우 list(['나', '는', '어제', '밥', '을' '먹다', '이다'])로 바꿔줌
docs2 = docs.copy()
# for i in tqdm(range(len(docs))):
#     docs2[i] = docs2[i].split()

In [6]:
#### 명사 빈도 상위 50개 단어 파일
topwords_df = pd.read_csv("../dataset/2022/본문_형태소분석/2월/나무문화_2월_형태소분석(명사).csv", encoding='UTF-8')
print(topwords_df.shape, topwords_df.columns)
topwords50 = topwords_df['명사'].to_list()[:50]
print(len(topwords50))
print(topwords50)

(135023, 2) Index(['명사', '명사빈도'], dtype='object')
50
['것', '수', '그', '이', '사람', '곳', '때', '우리', '말', '등', '생각', '나', '더', '내', '문화', '위', '의', '시간', '나무', '때문', '중', '일', '길', '전', '제', '사진', '안', '하나', '집', '위해', '책', '아이', '마음', '대한', '다른', '한국', '작품', '글', '가지', '마을', '공간', '시작', '를', '가장', '개', '모습', '속', '자신', '및', '알']


In [7]:
#### 중복 단어 제거하여 리스트 생성
unique_lines = [list(set(line)) for line in tqdm(docs2)]

100%|██████████| 700406/700406 [00:05<00:00, 129389.77it/s]


In [8]:
#### 중복 제거 잘 됐나 개수로 확인
print(len(docs[0]))
print(len(unique_lines[0]))

46
42


In [9]:
#### 전체 단어쌍 빈도 dict 형식으로 생성
freq_count = {}   #동시출현 빈도가 저장될 dict
for words in tqdm(unique_lines):
    # 전체 단어에 대한 동시 출현 빈도
    for i, a in enumerate(words):
        for b in words[i+1:]:
            if a == b: continue
            elif a>b:
                freq_count[b, a] = freq_count.get((b, a),0) + 1
            else :
                freq_count[a, b] = freq_count.get((a, b),0) + 1

100%|██████████| 700406/700406 [08:37<00:00, 1353.76it/s]


In [10]:
#### 딕셔너리를 데이터프레임에 담기
tt_freq_df=pd.DataFrame.from_dict(freq_count, orient='index')

In [11]:
#### 딕트to데이터프레임 저장
tt_freq_df.to_csv("../dataset/2022/단어동시출현행렬/2월/단어동시출현_딕셔너리_2월.csv", encoding='UTF-8')

In [12]:
#### dict형식을 컬럼명 지정해서 데이터프레임으로 만들기
list1 = [(tt_freq_df.index[i][0], tt_freq_df.index[i][1], tt_freq_df[0][i]) for i in tqdm(range(len(tt_freq_df)))]
freq_df2 = pd.DataFrame(list1, columns=["term1","term2","freq"])
print(freq_df2.shape)
freq_df2.head()

100%|██████████| 125273143/125273143 [14:36<00:00, 142958.82it/s]


(125273143, 3)


,term1,term2,freq
0,가,틀어놔가지고,1
1,가,그런지,709
2,가,랑,2211
3,가,나왔음,10
4,가,하는,31461


In [13]:
#### 컬럼명 지정한 dict_to_데이터프레임 저장
freq_df2.to_csv("../dataset/2022/단어동시출현행렬/2월/단어동시출현_df_2월.csv", index=False, encoding='UTF-8')

In [14]:
#### 50위 이내에 속한 단어들로 이루어진 Dataframe을 추출
list2 =[(freq_df2.loc[i][0], freq_df2.loc[i][1], freq_df2.loc[i][2]) for i in tqdm(range(len(freq_df2))) if (freq_df2['term1'][i] in topwords50) &(freq_df2['term2'][i] in topwords50)]
freq_df3 = pd.DataFrame(list2, columns=["term1","term2","freq"])
freq_df3

100%|██████████| 125273143/125273143 [26:10<00:00, 79774.23it/s]


,term1,term2,freq
0,곳,알,1079
1,곳,이,29148
2,곳,하나,1416
3,곳,집,1482
4,곳,사람,3301
...,...,...,...
1220,글,마을,240
1221,가지,마을,235
1222,마을,마음,264
1223,글,모습,191


In [15]:
#### 상위50위 단어들로 이루어진 단어쌍 데이터프레임 저장
freq_df3.to_csv("../dataset/2022/단어동시출현행렬/2월/단어동시출현_df_상위50_2월.csv", index=False, encoding='UTF-8')

In [16]:
#### 공출현빈도(1-mode matrix) 만들기
word_co_matx = np.zeros((50,50))
mat_idx = topwords50 # 단어동시출현행렬이니까 인덱스와 칼럼을 동일하게 지정
mat_col = topwords50
word_co_matx = pd.DataFrame(word_co_matx, index=mat_idx, columns=mat_col)
print(word_co_matx.shape)


#### 단어쌍의 첫 번째 단어가 인덱스와 같다 and 두 번째 단어가 칼럼과 같다 → 단어쌍빈도를 해당 셀 값에 삽입
for n in tqdm(range(len(freq_df3))):
    for idx in mat_idx:
        if idx == freq_df3['term1'][n]:
            for col in mat_col:
                if col == freq_df3['term2'][n]:
                    word_co_matx.loc[idx, col] = freq_df3['freq'][n]
                    word_co_matx.loc[col, idx] = freq_df3['freq'][n]

(50, 50)


100%|██████████| 1225/1225 [00:00<00:00, 1486.94it/s]


In [17]:
#### 1-mode 매트릭스 확인하기
word_co_matx

,것,수,그,이,사람,곳,때,우리,말,등,생각,나,더,내,문화,위,의,시간,나무,때문,중,일,길,전,제,사진,안,하나,집,위해,책,아이,마음,대한,다른,한국,작품,글,가지,마을,공간,시작,를,가장,개,모습,속,자신,및,알
것,0.0,23425.0,24063.0,85501.0,14409.0,6744.0,11375.0,10994.0,11870.0,6410.0,10257.0,11414.0,8030.0,8878.0,4360.0,6533.0,65449.0,4672.0,3872.0,7667.0,5712.0,8199.0,3050.0,5265.0,4269.0,2562.0,5890.0,4879.0,3115.0,4677.0,3363.0,3042.0,4966.0,5169.0,5479.0,2567.0,2716.0,2003.0,4725.0,1542.0,2398.0,3433.0,49592.0,4845.0,2592.0,3099.0,4863.0,5507.0,1624.0,5593.0
수,23425.0,0.0,13278.0,50885.0,8453.0,7946.0,6782.0,6468.0,6051.0,7416.0,5740.0,7551.0,5746.0,5544.0,4608.0,4426.0,45785.0,4431.0,3535.0,5190.0,4078.0,4925.0,2993.0,4201.0,3619.0,2762.0,3787.0,3089.0,2279.0,2911.0,2512.0,3026.0,3204.0,3321.0,3618.0,2008.0,2577.0,1533.0,3280.0,1953.0,4075.0,2164.0,38855.0,2938.0,2442.0,3035.0,3532.0,3059.0,2241.0,5434.0
그,24063.0,13278.0,0.0,48184.0,10108.0,6290.0,8070.0,6001.0,7450.0,4343.0,5160.0,7198.0,4104.0,5568.0,2659.0,4477.0,47180.0,2628.0,2559.0,4547.0,5175.0,4811.0,2355.0,3845.0,2883.0,1623.0,3917.0,3072.0,2328.0,3043.0,2118.0,1758.0,2943.0,3376.0,3132.0,1639.0,2337.0,1315.0,2814.0,1259.0,1562.0,2361.0,34339.0,2555.0,1859.0,2418.0,3834.0,3821.0,1195.0,3223.0
이,85501.0,50885.0,48184.0,0.0,32633.0,29148.0,27084.0,20159.0,22177.0,21042.0,19995.0,24368.0,16792.0,18572.0,11979.0,14040.0,154405.0,13358.0,11683.0,15218.0,14581.0,15547.0,9686.0,15024.0,12173.0,9702.0,14558.0,10201.0,9315.0,9833.0,8690.0,8702.0,10693.0,9868.0,10648.0,7275.0,8139.0,6340.0,9492.0,6836.0,8941.0,9272.0,112911.0,9517.0,8347.0,9643.0,10435.0,9505.0,5474.0,9305.0
사람,14409.0,8453.0,10108.0,32633.0,0.0,3301.0,4573.0,3621.0,5246.0,2166.0,4144.0,5066.0,2967.0,3417.0,1765.0,2313.0,24025.0,1735.0,1545.0,2585.0,2342.0,3430.0,1469.0,1917.0,1495.0,1348.0,2558.0,1752.0,1574.0,1861.0,1367.0,910.0,2275.0,1693.0,2908.0,936.0,865.0,871.0,1805.0,1045.0,1019.0,1252.0,18854.0,1593.0,1024.0,1436.0,1987.0,2560.0,505.0,1925.0
곳,6744.0,7946.0,6290.0,29148.0,3301.0,0.0,2922.0,2018.0,1676.0,2791.0,2080.0,2795.0,2119.0,2273.0,2370.0,1737.0,17485.0,2098.0,1875.0,1430.0,2515.0,1460.0,2431.0,2003.0,1805.0,2003.0,1939.0,1416.0,1482.0,1179.0,656.0,1816.0,1271.0,696.0,1751.0,890.0,654.0,656.0,847.0,2101.0,1848.0,962.0,13807.0,1818.0,1303.0,1469.0,1266.0,568.0,674.0,1079.0
때,11375.0,6782.0,8070.0,27084.0,4573.0,2922.0,0.0,3263.0,3615.0,2299.0,3312.0,4622.0,2514.0,3392.0,1242.0,1943.0,19528.0,1841.0,1414.0,1983.0,2107.0,2720.0,1221.0,2128.0,1952.0,1260.0,2392.0,1530.0,1567.0,1310.0,1100.0,1165.0,1682.0,1259.0,1578.0,922.0,735.0,664.0,1458.0,723.0,670.0,1566.0,17499.0,1369.0,1045.0,1192.0,1409.0,1384.0,520.0,1489.0
우리,10994.0,6468.0,6001.0,20159.0,3621.0,2018.0,3263.0,0.0,3078.0,1569.0,2915.0,2989.0,2279.0,2153.0,1993.0,1822.0,19308.0,1645.0,1195.0,1703.0,1508.0,2169.0,1177.0,1695.0,1077.0,905.0,2019.0,1362.0,1619.0,1553.0,1107.0,1330.0,1690.0,1494.0,1489.0,874.0,696.0,710.0,1301.0,605.0,660.0,929.0,14662.0,1062.0,796.0,1065.0,1860.0,1310.0,492.0,1726.0
말,11870.0,6051.0,7450.0,22177.0,5246.0,1676.0,3615.0,3078.0,0.0,1783.0,2613.0,3934.0,1861.0,2970.0,1062.0,1752.0,18356.0,1077.0,1002.0,1617.0,1609.0,2528.0,826.0,1620.0,1406.0,567.0,1965.0,1402.0,1038.0,1145.0,1073.0,879.0,1507.0,1166.0,1396.0,760.0,594.0,787.0,1339.0,392.0,442.0,908.0,13921.0,1002.0,778.0,781.0,1479.0,1643.0,422.0,1310.0
등,6410.0,7416.0,4343.0,21042.0,2166.0,2791.0,2299.0,1569.0,1783.0,0.0,1153.0,3054.0,1289.0,1875.0,2945.0,2228.0,21625.0,1232.0,2020.0,1219.0,2184.0,1670.0,1091.0,2541.0,2242.0,987.0,1261.0,952.0,1034.0,1527.0,982.0,867.0,614.0,1563.0,1111.0,1625.0,1319.0,601.0,1301.0,1030.0,1608.0,1067.0,15141.0,1039.0,1757.0,1023.0,1404.0,767.0,2715.0,782.0


In [18]:
#### 공출현빈도 데이터프레임 저장하기~~
#### ※ 인덱스가 저장돼야 하므로 index 파라미터 지정하지 말 것.
word_co_matx.to_csv("../dataset/2022/단어동시출현행렬/2월/공출현빈도_df_2월.csv", encoding='UTF-8')